In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import sympy as sym
import scipy as sci
import scipy.integrate as integrate


In [2]:
def r(x,q,c):
    if x>=1/2 and x<=q:
        return((q-x)+c)
    else:
        return(0)
'''
def K(q,c):
    return(integrate.quad(r,0,1, args=(q,c))[0])
'''
def K(q,c):
    return((1-4*c-4*q+8*c*q+4*q**2)/8)


print(K(0.8,0.3))
#print(K1(0.8,0.3))
print(K(0.8,0))
#print(K1(0.8,0))

0.135
0.04500000000000004


In [3]:
def f(x,q,c):
    return(r(x,q,c)/K(q,c))



def g0(x,q,c):
    if x>=0.5 and x<=q:
        return(x*f(x,q,c))
    else:
        return(x*f(1-x,q,c))


def g1(x,q,c):
    if x>=0.5 and x<=q:
        return((1-x)*f(x,q,c))
    else:
        return((1-x)*f(1-x,q,c))


def G0(x,q,c):
    return(integrate.quad(g0,0,x, args=(q,c))[0])

def G1(x,q,c):
    return(integrate.quad(g1,0,x, args=(q,c))[0])
'''

def G0(x,q,c):
    return(((3*(c+q)-2*x)*x**2)/(6*K(q,c)))


def G1(x,q,c):
    return((x*(6*(c+q)-3*(1+c+q)*x+2*x**2))/(6*K(q,c)))
'''

def muUpdate(mu,a,thold,barQ=0.8,c=0):
    if G0(thold,barQ,c)==0 or G0(thold,barQ,c)==1:
        return(mu)
    else:
        if (a==0):
            delta=(mu/(1-mu))*(1-G0(thold,barQ,c))/(1-G1(thold,barQ,c))
            return(delta/(1+delta))
        elif (a==1):
            delta=((mu)/(1-mu))*(G0(thold,barQ,c))/(G1(thold,barQ,c))
            return(delta/(1+delta))

def v(m,t0,t1,q):
    v_temp=(1-m)*(1+t0-t1)/(2*m-(2*m-1)*(1+t0-t1))
    if v_temp<1-q:
        return(1-q)
    elif v_temp>q:
        return(q)
    else: return(v_temp)


def dvtau(m,t0,t1):
    return((2*(1-m)*m)/((1-(2*m-1)*(t0-t1)))**2)

def focPi0(m,t0,t1,q,c):
    return(m*(1-G0(v(m,t0,t1,q),q,c))+(1-m)*(1-G1(v(m,t0,t1,q),q,c))-t0*(m*g0(v(m,t0,t1,q),q,c)+(1-m)*g1(v(m,t0,t1,q),q,c))*dvtau(m,t0,t1))


def focPi1(m,t0,t1,q,c):
    return(m*(G0(v(m,t0,t1,q),q,c))+(1-m)*(G1(v(m,t0,t1,q),q,c))-t1*(m*g0(v(m,t0,t1,q),q,c)+(1-m)*g1(v(m,t0,t1,q),q,c))*dvtau(m,t0,t1))

    

In [4]:
print(G0(0.7,0.8,5))
print(G1(0.7,0.8,5))
print(dvtau(0.6,0.9,0.5))
print(focPi0(0.5,0.2889,0.26659,0.8,0))
print(focPi1(0.5,0.2889,0.288556,0.8,0))

0.7549083063431649
0.9182308522022761
0.5671077504725898
-8.825905305287485e-05
0.019922562462607707


In [5]:
from mpmath import *
#from sympy import Symbol, solveset, S, erf, log, sqrt
mp.dps = 30; mp.pretty = True
findroot(sin, 3)

m=0.5
q=0.8
c=0


#f=[lambda x0,x1: focPi0(0.51,x0,x1,0.8,0)==0,
#    lambda x0,x1: focPi1(0.51,x0,x1,0.8,0)==0]

def findPrices(t0,t1,m,q,c,eps=0.1):
    print([t0,t1])
    barAlpha=m*q/(m*q+(1-m)*q)
    ubarAlpha=m*(1-q)/(m*(1-q)+(1-m)*q)
    if abs(focPi0(m,t0,t1,q,c))<eps and abs(focPi1(m,t0,t1,q,c))<(eps):
        return([t0,t1])
    else:
        if focPi0(m,t0,t1,q,c)<-eps:
            t0=t0-eps
        if focPi0(m,t0,t1,q,c)>eps:
            t0=t0+eps
        if focPi1(m,t0,t1,q,c)<-eps:
            t1=t1-eps
        if focPi1(m,t0,t1,q,c)>eps:
            t1=t1+eps
        findPrices(t0,t1,m,q,c,eps)
m=0.5
q=0.8
c=0

def findPricesBrute(m,q,c,freq=10):
    barAlpha=m*q/(m*q+(1-m)*(1-q)) 
    ubarAlpha=m*(1-q)/(m*(1-q)+(1-m)*q)
    grid0=np.linspace(ubarAlpha,barAlpha,freq)
    grid1=np.linspace(1-barAlpha,1-ubarAlpha,freq)
    t0=grid0[0]
    t1=grid1[0]
    absV=100000000
    for x in grid0:
        for y in grid1:
                  V=abs(focPi1(m,x,y,q,c))+abs(focPi0(m,x,y,q,c))
                  if V<absV:
                        absV=V
                        t0=x
                        t1=y
    #print(absV)                    
    return([t0,t1])


In [6]:
findPricesBrute(0.5,0.8,0.55)

[0.4666666666666667, 0.4666666666666667]

In [7]:
findPricesBrute(0.5,0.8,0.55,50)

[0.49387755102040815, 0.49387755102040815]

In [8]:
focPi0(0.5,0.3333333333333333,0.3333333333333333,0.8,0)

-0.05555555555555447

In [9]:
0.5*(1-G0(v(0.5,0.466667,0.466667,0.8),0.8,0.55))+0.5*(1-G1(v(0.5,0.466667,0.466667,0.8),0.8,0.55))

0.5000000000000002

In [10]:
def muUpdate(a,m,x,q,c):
    if a<1:
        delta=m*(1-G0(x,q,c))/((1-m)*(1-G1(x,q,c)))
    else:
        delta=m*G0(x,q,c)/((1-m)*G1(x,q,c))
    return(delta/(delta+1))    

In [11]:
print("+++++++++Fixed prices:+++++++++++")
n=0
mu=0.5
c=0.5
profitNDE=1
profitDE=0

while profitNDE>profitDE and n<10:
    print("Num consecutive zeros: "+str(n))
    print("********")
    profitDE=2*(mu*(1-q))/(mu*(1-q)+(1-mu)*q)-1
    t=[0,0]
    t0=t[0]
    t1=t[1]
    v1=v(mu,t0,t1,q)
    print("NDE prices are: "+str(t))
    profitNDE=(mu*(1-G0(v1,q,c))+(1-mu)*(1-G1(v1,q,c)))*t0
    prA=mu*G0(v1,q,c)+(1-mu)*G1(v1,q,c)
    mu=muUpdate(0,mu,v1,q,c)
    print("mu1:"+str(mu))
    print("Pr(a=1|h):"+str(prA))
    print("Pr(a=1|h,omega=1):"+str(G1(v1,q,c)))
    print("profit DE: "+str(profitDE))
    print("profit NDE: "+str(profitNDE))
    n=n+1
    print('+++++++++++++')
print(str(n))

print("+++++++++Fixed prices, c=0:+++++++++++")
n=0
mu=0.5
c=0
profitNDE=1
profitDE=0

while profitNDE>profitDE and n<10:
    print("Num consecutive zeros: "+str(n))
    print("********")
    profitDE=2*(mu*(1-q))/(mu*(1-q)+(1-mu)*q)-1
    t=[0,0]
    t0=t[0]
    t1=t[1]
    v1=v(mu,t0,t1,q)
    print("NDE prices are: "+str(t))
    profitNDE=(mu*(1-G0(v1,q,c))+(1-mu)*(1-G1(v1,q,c)))*t0
    prA=mu*G0(v1,q,c)+(1-mu)*G1(v1,q,c)
    mu=muUpdate(0,mu,v1,q,c)
    print("mu1:"+str(mu))
    print("Pr(a=1|h):"+str(prA))
    print("Pr(a=1|h,omega=1):"+str(G1(v1,q,c)))
    print("profit DE: "+str(profitDE))
    print("profit NDE: "+str(profitNDE))
    n=n+1
    print('+++++++++++++')
print(str(n))


+++++++++Fixed prices:+++++++++++
Num consecutive zeros: 0
********
NDE prices are: [0, 0]
mu1:0.6384615384614585
Pr(a=1|h):0.5
Pr(a=1|h,omega=1):0.6384615384614585
profit DE: -0.6000000000000001
profit NDE: 0.0
+++++++++++++
Num consecutive zeros: 1
********
NDE prices are: [0, 0]
mu1:0.6991873172135995
Pr(a=1|h):0.21184590458060332
Pr(a=1|h,omega=1):0.3442281441281096
profit DE: -0.3874538745388927
profit NDE: 0.0
+++++++++++++
Num consecutive zeros: 2
********
NDE prices are: [0, 0]
mu1:0.7327050792866195
Pr(a=1|h):0.11415753349221652
Pr(a=1|h,omega=1):0.21286167306357995
profit DE: -0.2649565458211599
profit NDE: 0.0
+++++++++++++
Num consecutive zeros: 3
********
NDE prices are: [0, 0]
mu1:0.7532947998325955
Pr(a=1|h):0.06931264082042607
Pr(a=1|h,omega=1):0.14100346304045505
profit DE: -0.1867348071511874
profit NDE: 0.0
+++++++++++++
Num consecutive zeros: 4
********
NDE prices are: [0, 0]
mu1:0.7667761965144687
Pr(a=1|h):0.045129739116611695
Pr(a=1|h,omega=1):0.09730936385884481

In [12]:

print("+++++++++Dynamic prices, c=0.5:+++++++++++")


n=0
mu=0.5
c=0.5
profitNDE=1
profitDE=0

while profitNDE>profitDE and n<10:
    print("Num consecutive zeros: "+str(n))
    print("********")
    profitDE=2*(mu*(1-q))/(mu*(1-q)+(1-mu)*q)-1
    t=findPricesBrute(mu,q,c)
    t0=t[0]
    t1=t[1]
    v1=v(mu,t0,t1,q)
    print("NDE prices are: "+str(t))
    profitNDE=(mu*(1-G0(v1,q,c))+(1-mu)*(1-G1(v1,q,c)))*t0
    prA=mu*G0(v1,q,c)+(1-mu)*G1(v1,q,c)
    mu=muUpdate(0,mu,v1,q,c)
    print("mu1:"+str(mu))
    print("Pr(a=1|h):"+str(prA))
    print("Pr(a=1|h,omega=1):"+str(G1(v1,q,c)))
    print("profit DE: "+str(profitDE))
    print("profit NDE: "+str(profitNDE))
    n=n+1
    print('+++++++++++++')
print(str(n))

print("+++++++++Dynamic prices, c=0:+++++++++++")


n=0
mu=0.5
c=0
profitNDE=1
profitDE=0

while profitNDE>profitDE and n<10:
    print("Num consecutive zeros: "+str(n))
    print("********")
    profitDE=2*(mu*(1-q))/(mu*(1-q)+(1-mu)*q)-1
    t=findPricesBrute(mu,q,c)
    t0=t[0]
    t1=t[1]
    v1=v(mu,t0,t1,q)
    print("NDE prices are: "+str(t))
    profitNDE=(mu*(1-G0(v1,q,c))+(1-mu)*(1-G1(v1,q,c)))*t0
    prA=mu*G0(v1,q,c)+(1-mu)*G1(v1,q,c)
    mu=muUpdate(0,mu,v1,q,c)
    print("mu1:"+str(mu))
    print("Pr(a=1|h):"+str(prA))
    print("Pr(a=1|h,omega=1):"+str(G1(v1,q,c)))
    print("profit DE: "+str(profitDE))
    print("profit NDE: "+str(profitNDE))
    n=n+1
    print('+++++++++++++')
print(str(n))

+++++++++Dynamic prices, c=0.5:+++++++++++
Num consecutive zeros: 0
********
NDE prices are: [0.4666666666666667, 0.4666666666666667]
mu1:0.6384615384614585
Pr(a=1|h):0.5
Pr(a=1|h,omega=1):0.6384615384614585
profit DE: -0.6000000000000001
profit NDE: 0.23333333333333334
+++++++++++++
Num consecutive zeros: 1
********
NDE prices are: [0.622782164497053, 0.4405196558562468]
mu1:0.7354133245737858
Pr(a=1|h):0.36394648492577397
Pr(a=1|h,omega=1):0.5345134671689522
profit DE: -0.3874538745388927
profit NDE: 0.3961227848538854
+++++++++++++
Num consecutive zeros: 2
********
NDE prices are: [0.6919249814383439, 0.3080750185616561]
mu1:0.812875368410179
Pr(a=1|h):0.33213655401562986
Pr(a=1|h,omega=1):0.5276644181691409
profit DE: -0.18003153844443331
profit NDE: 0.46211140246608373
+++++++++++++
Num consecutive zeros: 3
********
NDE prices are: [0.7094892798456167, 0.19607380381183875]
mu1:0.8644605610853067
Pr(a=1|h):0.2605696540221111
Pr(a=1|h,omega=1):0.4644105730023387
profit DE: 0.0412308

In [26]:
eps=0.01
q=0.8
c=1
print("++++"+str(c)+":+++++")
print('G1(1-q+eps,q,c): '+str(G1(1-q+eps,q,c)))
print('G0(1-q+eps,q,c): '+str(G0(1-q+eps,q,c)))


eps=0.01
q=0.8
c=0.5
print("++++"+str(c)+":+++++")
print('G1(1-q+eps,q,c): '+str(G1(1-q+eps,q,c)))
print('G0(1-q+eps,q,c): '+str(G0(1-q+eps,q,c)))



eps=0.01
q=0.8
c=0.01
print("++++"+str(c)+":+++++")
print('G1(1-q+eps,q,c): '+str(G1(1-q+eps,q,c)))
print('G0(1-q+eps,q,c): '+str(G0(1-q+eps,q,c)))



eps=0.01
q=0.8
c=0
print("++++"+str(c)+":+++++")
print('G1(1-q+eps,q,c): '+str(G1(1-q+eps,q,c)))
print('G0(1-q+eps,q,c): '+str(G0(1-q+eps,q,c)))




++++1:+++++
G1(1-q+eps,q,c): 0.02315845403861508
G0(1-q+eps,q,c): 0.00597198101064538
++++0.5:+++++
G1(1-q+eps,q,c): 0.020588034355137463
G0(1-q+eps,q,c): 0.005309402046778938
++++0.01:+++++
G1(1-q+eps,q,c): 0.0024826389064921996
G0(1-q+eps,q,c): 0.0006423612367346774
++++0:+++++
G1(1-q+eps,q,c): 0.0008814811588269771
G0(1-q+eps,q,c): 0.0002296296490526259


In [ ]:
!pip3 install --user sympy 

  Using cached https://files.pythonhosted.org/packages/4d/a7/25d5d6b3295537ab90bdbcd21e464633fb4a0684dd9a065da404487625bb/sympy-1.5-py2.py3-none-any.whl
